In [133]:
import pandas as pd
import sqlite3

In [134]:
connect_db = sqlite3.connect('checking-logs.sqlite')
query = 'PRAGMA table_info(test);'
pd.io.sql.read_sql(query, connect_db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [135]:
query = 'select * from test'
pd.io.sql.read_sql(query, connect_db).head()

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447


In [136]:
query = 'PRAGMA table_info(deadlines);'
pd.io.sql.read_sql(query, connect_db)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [137]:
query = 'select * from deadlines'
pd.io.sql.read_sql(query, connect_db, parse_dates=['deadlines'])

,index,labs,deadlines
0,0,laba04,2020-04-26 23:59:59
1,1,laba04s,2020-04-26 23:59:59
2,2,laba05,2020-05-03 23:59:59
3,4,laba06,2020-05-24 23:59:59
4,5,laba06s,2020-05-24 23:59:59
5,3,project1,2020-05-16 23:59:59


In [138]:
query = '''select uid, min(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as delta from test left join deadlines on test.labname = deadlines.labs 
           where labname <> 'project1' '''
          #  where checker.status = 'ready' and checker.uid like 'user_%' and checker.numTrials = 1 and checker.labname in ('laba04', 'laba04s', 'laba05', 'laba06','laba06s', 'project1')
          #  and (pageviews.datetime = (select MIN(pageviews.datetime) from pageviews where pageviews.uid=checker.uid) or pageviews.datetime IS NULL) '''
df_min = pd.io.sql.read_sql(query, connect_db)
df_min

,uid,delta
0,user_30,-202


In [139]:
query = '''select uid, max(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as delta from test left join deadlines on test.labname = deadlines.labs 
           where labname <> 'project1' '''
          #  where checker.status = 'ready' and checker.uid like 'user_%' and checker.numTrials = 1 and checker.labname in ('laba04', 'laba04s', 'laba05', 'laba06','laba06s', 'project1')
          #  and (pageviews.datetime = (select MIN(pageviews.datetime) from pageviews where pageviews.uid=checker.uid) or pageviews.datetime IS NULL) '''
df_max = pd.io.sql.read_sql(query, connect_db)
df_max

,uid,delta
0,user_25,-2


In [140]:
query = '''select uid, avg(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as delta from test left join deadlines on test.labname = deadlines.labs 
           where labname <> 'project1' '''
          #  where checker.status = 'ready' and checker.uid like 'user_%' and checker.numTrials = 1 and checker.labname in ('laba04', 'laba04s', 'laba05', 'laba06','laba06s', 'project1')
          #  and (pageviews.datetime = (select MIN(pageviews.datetime) from pageviews where pageviews.uid=checker.uid) or pageviews.datetime IS NULL) '''
df_avg = pd.io.sql.read_sql(query, connect_db)
df_avg

,uid,delta
0,user_17,-89.125


In [141]:
query = '''select test.uid, avg(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS Integer)) as avg_diff, views.pageviews
           from test left join deadlines on test.labname = deadlines.labs left join (select pageviews.uid, count(1) as pageviews from pageviews group by pageviews.uid) as views on views.uid = test.uid
           where labname <> 'project1' group by test.uid
           '''
# query = ''' select uid, count(1) as pageviews from pageviews group by uid '''
          #  where checker.status = 'ready' and checker.uid like 'user_%' and checker.numTrials = 1 and checker.labname in ('laba04', 'laba04s', 'laba05', 'laba06','laba06s', 'project1')
          #  and (pageviews.datetime = (select MIN(pageviews.datetime) from pageviews where pageviews.uid=checker.uid) or pageviews.datetime IS NULL) '''
views_diff = pd.io.sql.read_sql(query, connect_db)
views_diff
# uid, avg_diff, pageviews

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [142]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [143]:
connect_db.close()